In [46]:
# Env
import os
import pandas as pd
import numpy as np

workdir = r'./dataset/2023.12.8_Human early development'
resultsdir = os.path.join(workdir, 'results')
gene_list = pd.read_excel(os.path.join(workdir, 'Human early development_gene list_v4_ML_2311.xlsx'), sheet_name='UNIQUE GENE LIST')
gene_list = [_.upper() for _ in gene_list['Unique gene list'].unique()]

In [47]:
result = pd.DataFrame()
for dir in os.listdir(os.path.join(resultsdir)):
    try:
        df = pd.read_excel(os.path.join(resultsdir, dir, "probes_wanted.xlsx"), index_col=0)
        # df = df[df.columns[1:]]
        result = pd.concat([result, df])
    except:
        continue

# result["gene_name"] = [_ if _ in gene_list else None for _ in result["gene_name"]]
result["gene_name"] = pd.Categorical(
    [_.upper() for _ in result["gene_name"]], categories=gene_list, ordered=True
)
result = result.sort_values(["gene_name", "pos_on_seq"])
result.drop_duplicates(subset=["binding"], keep="first", inplace=True)
# result = result.set_index("accession")

result.to_excel(os.path.join(workdir, 'gene_binding_site.xlsx'))

to_search = [gene for gene in gene_list if len(result[result["gene_name"] == gene]) < 1]
with open(os.path.join(workdir, "to_search.txt"), "w") as f:
    for line in to_search:
        f.write(line + "\n")